In [ ]:
!git clone https://github.com/VahidZee/DRMADE.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%tensorflow_version 2.x

In [ ]:
%load_ext tensorboard

In [ ]:
cd DRMADE/

In [ ]:
from src.model.drmade import trainers
from torchvision import datasets

In [ ]:
hyper_parameters = {
    'latent_size': 128,
    
    'pgd/eps': 0.,
    'pgd/iterations': 1,
    'pgd/alpha': 0.05,
    'pgd/randomize': False,
    
    'pgd_latent/eps':0.,
    'pgd_latent/iterations': 1,
    'pgd_latent/alpha': 0.05,
    'pgd_latent/randomize': False,

    'base_lr': 3e-2,
    'lr_schedule': 40,
    'normal_classes':[8],
    'dataset': datasets.MNIST,

    'max_epoch': 1025,
    'embedding_interval': 512,
    'evaluation_interval': 32,
    'validation_interval':32,
    'save_interval': 32,

    'output_root': './output/',
}

In [ ]:
trainer = trainers.RobustAutoEncoderPreTrainer(hparams=hyper_parameters)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
%tensorboard --logdir ./runs

In [ ]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1
! unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

In [ ]:
get_ipython().system_raw(
    'tensorboard --logdir="{}" --host 0.0.0.0 --port 6006 &'.format(config.runs_dir)
)

In [ ]:
get_ipython().system_raw('../ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"